**Rename Files by removing the first 23 characters**

```
for file in *; do mv $file `echo $file | cut -c23-`; done
```

**Unzip all the files**

```
bunzip2 *.bz2
```

**Create the alignment.csv file**

```
(echo 'samplename,description'; for f in raw/*fastq; do readlink -f $f | perl -pe 's/(.*?_(S[0-9]+)_.*)/\1,\2/'; done) > to_merge.csv
```

**Merge multiple files into a single file**

```
#!/bin/sh
#SBATCH -p priority
#SBATCH -J merge
#SBATCH -o run.o
#SBATCH -e run.e
#SBATCH -t 0-12:00
#SBATCH --cpus-per-task=20
#SBATCH --mail-type=END         # Type of email notification- BEGIN,END,FAIL,ALL
#SBATCH --mail-user=ajitj_nirmal@dfci.harvard.edu   # Email to which notifications will be sent

module load bcbio/latest
cd raw
bcbio_prepare_samples.py --out merged --csv to_merge.csv
```

**Download the reference genome**

```
mkdir reference
cd reference
wget ftp://ftp.ensembl.org/pub/release-96/fasta/homo_sapiens/cdna/Homo_sapiens.GRCh38.cdna.all.fa.gz
wget ftp://ftp.ensembl.org/pub/release-96/gtf/homo_sapiens/Homo_sapiens.GRCh38.96.gtf.gz
gunzip *
```

**Prepare the O2.yaml file**

```
cd ..
vim O2.yaml

details:
  - analysis: RNA-seq
    genome_build: hg38
    algorithm:
      transcriptome_fasta: /n/scratch2/ajit/jain_rnaseq/reference/Homo_sapiens.GRCh38.cdna.all.fa
      transcriptome_gtf: //n/scratch2/ajit/jain_rnaseq/reference/Homo_sapiens.GRCh38.96.gtf
      aligner: hisat2
      strandedness: unstranded
      tools_on: [bcbiornaseq]
      bcbiornaseq:
          organism: homo sapiens
upload:
  dir: ../final
```

**Intiate bcBio**

```
module load bcbio/latest
unset PYTHONPATH
bcbio_nextgen.py -w template O2.yaml alignment.csv raw_files/
```

**Create Submission script to O2**

```
#!/bin/sh
#SBATCH -p priority
#SBATCH -J pickseq
#SBATCH -o run.o
#SBATCH -e run.e
#SBATCH -t 10-00:00
#SBATCH --cpus-per-task=20
#SBATCH --mem=150G
#SBATCH --mail-type=END         # Type of email notification- BEGIN,END,FAIL,ALL
#SBATCH --mail-user=ajitj_nirmal@dfci.harvard.edu   # Email to which notifications will be sent

export PATH=/n/app/bcbio/tools/bin:$PATH
bcbio_nextgen.py ../config/alignment.yaml \
    -n 24 -t local
```


**Submit job to O2 for processing**

```
cp submit_bcbio.sh alignment/work
cd alignment/work
sbatch submit_bcbio.sh
```